In [2]:
import pandas as pd
import sqlite3




## building database for Flask app

In [21]:
df = pd.read_csv("../static/data/slc_daily_pm2.5_pm10_2016to2024.csv")
df = df.sort_values(['date'], ascending = False)
latest_sensor_df = df.groupby(['sensor_id']).first().reset_index()
latest_sensor_df = latest_sensor_df[['sensor_id','latitude','longitude','pm2.5_atm']]

In [23]:
# Define the SQLite database name
database_name = '../static/data/test_sensor.db'

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(database_name)

# Convert the DataFrame to a SQL table within the database
# Replace 'data_table' with your desired table name
# Correct usage with a string table name
latest_sensor_df.to_sql('latest_sensor_data', conn, if_exists='replace', index=False)


# Close the connection to the database
conn.close()